In [1]:
import os
import cv2 
import numpy as np
import pandas as pd
import skimage
from skimage import measure
from collections import Counter
import matplotlib.pyplot as plt

In [2]:
def crop(target):
    img=cv2.imread(target, cv2.IMREAD_GRAYSCALE).astype(np.uint8)

    thr, img_thr= cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img_test=measure.label(img_thr, connectivity = 1) # We get binary regions of the image
    props = measure.regionprops(img_test) 
    areas=[prop.area for prop in props] #We get the area for each region
    ind_max_area=np.argmax(areas)+1 #We select the maximum area region (body)

    boxes = [prop.bbox for prop in props]
    box = boxes[ind_max_area-1]
    img_new = img[box[0]:box[2], box[1]:box[3]]
    img_new= cv2.resize(img_new, (512,512))
    return img_new

def split_target_dir_and_crop(target_dir,output_dir):
    target_list=[target_dir+os.sep+file for file in os.listdir(target_dir)]
    for target in target_list:
        img_new= crop(target)     
        dst=target.replace(target_dir,output_dir)
        dst_dir=os.path.split(dst)[0]
        if not os.path.exists(dst_dir):
            os.makedirs(dst_dir)
        cv2.imencode('.jpg', img_new)[1].tofile(dst)
    print(f'Target list done with {len(target_list)} items')

In [ ]:
#Cropping and segmentation of patient's images

lista_percorsi = os.listdir('./CT/')

for i,path in enumerate(lista_percorsi):
    if path.find('Patient') >= 0:
        cartella2=os.listdir('CT/'+path+'/')[0]
        source = 'CT/'+path+'/'+cartella2+'/'+os.listdir('CT/'+path+'/'+cartella2+'/')[0]
        path=path.replace(' ','_')
        dst = 'CT_segmentate3/'+path+'/'
        split_target_dir_and_crop(source, dst)

In [ ]:
#Cropping and segmentation of nCT, pCT, NiCT images

source = './nCT_pCT_NiCT/nCT_pCT_NiCT/nCT/'
dst = './nCT_pCT_NiCT_segmented/nCT_pCT_NiCT/nCT/'
split_target_dir_and_crop(source, dst)
source = './nCT_pCT_NiCT_segmentedz/nCT_pCT_NiCT/pCT/'
dst = './nCT_pCT_NiCT_segmented/nCT_pCT_NiCT/pCT/'
split_target_dir_and_crop(source, dst)
source = './nCT_pCT_NiCT/nCT_pCT_NiCT/NiCT/'
dst = './nCT_pCT_NiCT_segmented/nCT_pCT_NiCT5/NiCT/'
split_target_dir_and_crop(source, dst)